# LangChain ReAct Agent

In this notebook we demonstrate LangChain’s ReAct agent enhanced tools. Along with a calculator for arithmetic, we have added:
- **Joke Teller**: Returns a random joke.
- **Sentiment Analyzer**: Gives a basic sentiment evaluation (positive, negative, or neutral) based on the input text.
- **Weather Forecast**: Provides a simulated weather forecast for a mentioned city.



In [ ]:
import os
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import Tool
from langchain.chains import LLMMathChain
from langchain_community.chat_models import ChatOpenAI

# Set your OpenAI API key

from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize the language model with a temperature of 0 for deterministic output
#llm = OpenAI(temperature=0, )
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key,model_name="gpt-4o")

Below, we define four tools:
1. **Calculator**: Evaluates arithmetic expressions.
2. **Joke Teller**: Returns a random joke from a predefined list.
3. **Sentiment Analyzer**: Performs a simple sentiment analysis based on the occurrence of positive and negative words.
4. **Weather Forecast**: Simulates weather information based on the city mentioned in the query.

In [ ]:
#using a predefined function from langchain
math_chain = LLMMathChain.from_llm(OpenAI(temperature=0))
calculator_tool = Tool(
    name="Calculator",
    func=math_chain.run,
    description="Useful for answering math-related questions."
)

# Joke Teller tool
def tell_joke(query: str):
    """
    Returns a random joke.
    The input query is ignored; this tool always returns a random joke.
    """
    import random
    jokes = [
        "Why don't scientists trust atoms? Because they make up everything!",
        "I told my computer I needed a break, and it said 'No problem – I'll go to sleep.'",
        "Why did the math book look sad? Because it had too many problems."
    ]
    return random.choice(jokes)

joke_tool = Tool(
    name='Joke Teller',
    func=tell_joke,
    description='Returns a random joke to brighten your day.'
)

# Sentiment Analyzer tool
def analyze_sentiment(text: str):
    """
    Performs a basic sentiment analysis on the provided text.
    It counts positive and negative words to determine the overall sentiment.
    """
    positive_words = ["happy", "good", "great", "fantastic", "awesome"]
    negative_words = ["sad", "bad", "terrible", "awful", "horrible"]
    text_lower = text.lower()
    pos_count = sum(text_lower.count(word) for word in positive_words)
    neg_count = sum(text_lower.count(word) for word in negative_words)
    if pos_count > neg_count:
        sentiment = "Positive"
    elif neg_count > pos_count:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    return f"Sentiment: {sentiment} (Positive: {pos_count}, Negative: {neg_count})"

sentiment_tool = Tool(
    name='Sentiment Analyzer',
    func=analyze_sentiment,
    description='Analyzes the sentiment of the provided text, returning positive, negative, or neutral feedback.'
)

# Weather Forecast tool
def weather_forecast(query: str):
    """
    Simulates a weather forecast based on the provided city name.
    If a recognized city is mentioned, a preset forecast is returned; otherwise, a random forecast is provided.
    """
    import random
    cities = {
        "new york": "Cloudy with a chance of rain.",
        "london": "Overcast with light showers.",
        "paris": "Sunny with scattered clouds.",
        "tokyo": "Rainy with moderate winds."
    }
    query_lower = query.lower()
    for city, forecast in cities.items():
        if city in query_lower:
            return f"Weather in {city.title()}: {forecast}"
    random_forecasts = [
        "Sunny and bright!",
        "Partly cloudy with occasional rain.",
        "Cold and windy.",
        "Warm and humid."
    ]
    return "Weather: " + random.choice(random_forecasts)

weather_tool = Tool(
    name='Weather Forecast',
    func=weather_forecast,
    description='Provides a simulated weather forecast for a given city.'
)


## 3. Initializing the ReAct Agent with Multiple Tools

We now initialize the ReAct agent using all four tools. The agent will analyze incoming queries and determine whether to perform calculations, tell a joke, analyze sentiment, or simulate weather—sometimes even combining these tools as needed.


In [ ]:
# List all the interesting tools
tools = [calculator_tool, joke_tool, sentiment_tool, weather_tool]

# Initialize the agent using the ReAct framework
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [ ]:
# Testing one of the tools with a sample query
test_result = agent.run("Calculate 12 * (5 + 3)")
print('Test Query - Calculator:', test_result)


## 4. Interactive Section

Use the interactive text box below to type your queries. Examples:
- "Calculate 5 + 7" to perform a math calculation.
- "Tell me a joke" to get a random joke.
- "I feel so happy today!" to analyze sentiment.
- "What's the weather in London?" to receive a weather forecast.

The agent will decide which tool(s) to use based on your query.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create an input text box widget
query_box = widgets.Text(
    value='',
    placeholder='Enter your query here...',
    description='Query:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')  # Adjust width for better appearance
)

# Create a submit button
submit_button = widgets.Button(
    description="Submit",
    button_style="primary",
    tooltip="Click to submit your query",
    icon="paper-plane",
    layout=widgets.Layout(width='15%')
)

# Output area for displaying results
output_area = widgets.Output()

# Function to handle query submission
def on_query_submit(change=None):
    query = query_box.value.strip()  # Get user input
    query_box.value = ''  # Clear text box after submission
    
    with output_area:
        clear_output()  # Clear previous output
        if query:
            print(f"Query: {query}\n")
            try:
                result = agent.run(query)  # Use error-handling agent
                print("Result:", result)
            except Exception as e:
                print("Error:", e)
        else:
            print("⚠️ Please enter a query.")

# Bind the text box submit event
query_box.on_submit(on_query_submit)

# Bind the button click event
submit_button.on_click(on_query_submit)

# Display UI components in a horizontal layout
display(widgets.HBox([query_box, submit_button]), output_area)